In [1]:
# Kiểm tra GPU lại sau khi có
!nvidia-smi


Wed Apr  9 09:04:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!apt-get install unrar


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [4]:
!unrar x -o+ "/content/drive/MyDrive/PBL5/Dataset_ConTrung.rar" "/content/dataset/"

Streaming output truncated to the last 5000 lines.
Extracting  /content/dataset/Dataset_ConTrung/classification/val/3/02111.jpg      93%  OK 
Extracting  /content/dataset/Dataset_ConTrung/classification/val/3/02114.jpg      93%  OK 
Extracting  /content/dataset/Dataset_ConTrung/classification/val/3/02119.jpg      93%  OK 
Extracting  /content/dataset/Dataset_ConTrung/classification/val/3/02127.jpg      93%  OK 
Extracting  /content/dataset/Dataset_ConTrung/classification/val/3/02131.jpg      93%  OK 
Extracting  /content/dataset/Dataset_ConTrung/classification/val/3/02132.jpg      93%  OK 
Extracting  /content/dataset/Dataset_ConTrung/classification/val/3/02133.jpg      93%  OK 
Extracting  /content/dataset/Dataset_ConTrung/classification/val/3/02173.jpg      93%  OK 
Extracting  /content/dataset/Dataset_ConTrung/classification/val/3/02182.jpg      93%  OK 
Extracting  /content/dataset/Dataset_ConTrung/cla

In [5]:
!ls /content/dataset/

Dataset_ConTrung


In [6]:
!pip install timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [7]:
import numpy as np
import pandas as pd
import os
import random
from tqdm import tqdm
from textwrap import wrap

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import timm

In [8]:
f = open('/content/dataset/Dataset_ConTrung/classes.txt')
label = []
name = []
for line in f.readlines():
    label.append(int(line.split()[0]))
    name.append(' '.join(line.split()[1:]))
classes = pd.DataFrame([label, name]).T
classes.columns = ['label','name']
classes

,label,name
0,1,rice leaf roller
1,2,rice leaf caterpillar
2,3,paddy stem maggot
3,4,asiatic rice borer
4,5,yellow rice borer
...,...,...
97,98,Chlumetia transversa
98,99,Mango flat beak leafhopper
99,100,Rhytidodera bowrinii white
100,101,Sternochetus frigidus


In [9]:
train_df = pd.read_csv('/content/dataset/Dataset_ConTrung/train.txt',sep=' ',header=None, engine='python')
train_df.columns = ['image_path','label']

test_df = pd.read_csv('/content/dataset/Dataset_ConTrung/test.txt',sep=' ',header=None, engine='python')
test_df.columns = ['image_path','label']

val_df = pd.read_csv('/content/dataset/Dataset_ConTrung/val.txt',sep=' ',header=None, engine='python')
val_df.columns = ['image_path','label']

train_df.head()

,image_path,label
0,00002.jpg,0
1,00003.jpg,0
2,00005.jpg,0
3,00006.jpg,0
4,00008.jpg,0


In [10]:
TRAIN_DIR = '/content/dataset/Dataset_ConTrung/classification/train'
TEST_DIR = '/content/dataset/Dataset_ConTrung/classification/test'
VAL_DIR = '/content/dataset/Dataset_ConTrung/classification/val'
LR = 1e-5
BATCH_SIZE = 8
EPOCH = 5

device = torch.device('cuda')

In [11]:
fig, axs = plt.subplots(10,11,figsize=(30,30))
images = []
for i in classes.label:
    random_img = random.choice(train_df[train_df.label==i-1].image_path.values)
    label = classes.name[i-1]
    img = plt.imread(os.path.join(TRAIN_DIR,str(i-1),random_img))
    images.append(img)

[ax.imshow(image) for image,ax in zip(images,axs.ravel())]
[ax.set_title("\n".join(wrap(label,20))) for label,ax in zip(list(classes.name),axs.ravel())]
[ax.set_axis_off() for ax in axs.ravel()]
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [12]:
!pip install torchviz

In [13]:
import torch.nn as nn

class InsectModel(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.3):
        super(InsectModel, self).__init__()
        self.num_classes = num_classes
        self.model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)

        # Thêm Dropout vào lớp cuối
        self.dropout = nn.Dropout(p=dropout_rate)

        # Thêm lớp Fully Connected với Dropout
        in_features = self.model.head.in_features  # Kích thước đầu ra của ViT
        self.model.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(512, num_classes)
        )

    def forward(self, image):
        return self.model(image)

In [14]:
def train_transform():
    return A.Compose([
        A.HorizontalFlip(),
        A.RandomRotate90(),
        A.RandomBrightnessContrast(),
        A.Resize(224, 224),
        ToTensorV2()])

def valid_transform():
    return A.Compose([
        A.Resize(224,224),
        ToTensorV2()])

In [15]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [16]:
class InsectDataset(Dataset):
    def __init__(self, image, image_dir, transforms=None):
        self.image_info = image
        self.transforms = transforms
        self.imgdir = image_dir
    def __len__(self):
        return self.image_info.shape[0]
    def __getitem__(self, index):
        image_info = self.image_info[index]
        image = cv2.imread(os.path.join(self.imgdir,str(image_info[1]),image_info[0]),cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.

        if self.transforms is not None:
            image = self.transforms(image = image)['image']

        label = image_info[1]

        image = torch.as_tensor(image, dtype=torch.float32)
        label = torch.as_tensor(label, dtype=torch.long)
        return image, label

In [17]:
train_dataset = InsectDataset(image=train_df.values,
                              image_dir=TRAIN_DIR,
                              transforms=train_transform())
train_data_loader = DataLoader(train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=2)
val_dataset = InsectDataset(image=val_df.values,
                            image_dir=VAL_DIR,
                            transforms=valid_transform())
val_data_loader = DataLoader(val_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True,
                             num_workers=2)

In [18]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.loss = 0
        self.correct = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, loss,correct, n=1):
        self.loss = loss
        self.correct += correct
        self.sum += loss * n
        self.count += n

        self.avg = self.sum / self.count
        self.acc = self.correct / self.count

class Accuracy(object):
    def __init__(self):
        self.reset

In [19]:
# Định nghĩa các danh sách để lưu giá trị
train_losses = []
val_losses = []
train_accs = []
val_accs = []

def train_fn(data_loader, model, criterion, device, optimizer, epoch):
    model.train()
    criterion.train()

    summary = AverageMeter()
    tk0 = tqdm(data_loader, total=len(data_loader))
    for step, (images, labels) in enumerate(tk0):
        images = images.to(device, non_blocking = True).float()
        labels = labels.to(device, non_blocking = True).long()

        output = model(images)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = output.softmax(1).argmax(1)
        correct = (preds == labels).sum().item()

        summary.update(loss.item(),correct, BATCH_SIZE)



        tk0.set_postfix(loss=summary.avg, acc=summary.acc, epoch=epoch+1)

    return summary

def eval_fn(data_loader, model, criterion, device, epoch):
    model.eval()
    criterion.eval()

    summary = AverageMeter()
    tk0 = tqdm(data_loader, total=len(data_loader))
    with torch.no_grad():
        for step, (images, labels) in enumerate(tk0):
            images = images.to(device, non_blocking = True).float()
            labels = labels.to(device, non_blocking = True).long()

            output = model(images)
            loss = criterion(output, labels)

            preds = output.softmax(1).argmax(1)
            correct = (preds == labels).sum().item()

            summary.update(loss.item(), correct, BATCH_SIZE)


            tk0.set_postfix(loss=summary.avg, acc=summary.acc, epoch=epoch+1)
    return summary

In [20]:
import os
import torch
import torch.nn as nn
from tqdm import tqdm

# Tắt console của Weights & Biases (nếu không sử dụng)
os.environ['WANDB_CONSOLE'] = 'off'



def run():
    global train_losses, val_losses, train_accs, val_accs  # Để truy cập các danh sách toàn cục
    import torch.optim as optim

    model = InsectModel(num_classes=102)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR,  weight_decay=1e-4)
    best_loss = 10**5

    for epoch in range(0, EPOCH):
        # Huấn luyện
        train_summary = train_fn(train_data_loader, model, criterion, device, optimizer, epoch)
        # Đánh giá
        val_summary = eval_fn(val_data_loader, model, criterion, device, epoch)

        # Lưu mô hình nếu val_loss cải thiện
        if val_summary.avg < best_loss:
            best_loss = val_summary.avg
            torch.save(model.state_dict(), f'vit_best.pth')
        train_losses.append(train_summary.avg)
        train_accs.append(train_summary.acc)
        val_losses.append(val_summary.avg)
        val_accs.append(val_summary.acc)

        # In kết quả
        print(f'Epoch {epoch+1:03}: | Train Loss: {train_summary.avg:.5f} | Train Acc: {train_summary.acc:.5f} | '
              f'Val Loss: {val_summary.avg:.5f} | Val Acc: {val_summary.acc:.5f}')
    return model


In [21]:
myModel = run()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

  1%|          | 58/5637 [00:22<35:25,  2.62it/s, acc=0.0431, epoch=1, loss=4.48]


KeyboardInterrupt: 

In [ ]:
model = InsectModel(num_classes=102)
model.load_state_dict(torch.load("./vit_best.pth"))
images, labels = next(iter(val_data_loader))
preds = model(images).softmax(1).argmax(1)

fig, axs = plt.subplots(2,4,figsize=(13,8))
[ax.imshow(image.permute((1,2,0))) for image,ax in zip(images,axs.ravel())]
[ax.set_title("\n".join(wrap(f'Accutual: {classes.name[label.item()]} Predicted: {classes.name[pred.item()]}',30)),color = 'g' if label.item()==pred.item() else 'r') for label,pred,ax in zip(labels,preds,axs.ravel())]
[ax.set_axis_off() for ax in axs.ravel()]
plt.show()

NameError: name 'InsectModel' is not defined

In [ ]:
def plot_metrics():
    epochs = range(1, EPOCH + 1)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss', marker='o')
    plt.plot(epochs, val_losses, label='Val Loss', marker='o')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accs, label='Train Accuracy', marker='o')
    plt.plot(epochs, val_accs, label='Val Accuracy', marker='o')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()
plot_metrics()